# ML Pipeline Preparation


### 1. Import libraries and load data from database.


In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy.engine import create_engine

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('data', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message']
Y = df.iloc[:,-36:]

In [4]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

### 2. Write a tokenization function to process your text data


In [5]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#libraries for the tokenize function
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) #removing all punctuations
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        #clean_tokens = [w for w in clean_tokens if w not in stopwords.words("english")]
        
    return clean_tokens
    

In [7]:
#checking how tokenize() works
for text in X[:5]:
    tokens = tokenize(text)
    print(text)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'of', 'haiti', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
#libraries to build the ML model
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier


In [9]:
Y.shape[1] #n_classes for the multilabel classfication - labels

36

In [10]:
X.shape

(26216,)

In [11]:
pipeline = Pipeline([
            ('vect', CountVectorizer(token_pattern=None,tokenizer = tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2)


In [13]:
#saving the split datasets to

In [14]:
X_train.shape


(20972,)

In [15]:
X_train

18499    This year Ireland became the first country in ...
6563         I don't have any shelter, search one for me. 
16762    But if forecasts for heavy rain upriver in Mal...
11632    On My Way To SantiagO with the team . PhOtOshO...
18440    In the light of the results of the Ministry of...
                               ...                        
22412    (http://reliefweb.int/https://www.msb.se/en/To...
18774    They are in desperate need of clothes, domesti...
2625     Urgent help in Carrefour : Mahotiere 75 rue Ne...
15288    In Bihar, fresh areas were submerged by overfl...
4605                  thank you for the help. Good night. 
Name: message, Length: 20972, dtype: object

In [16]:
y_train.shape

(20972, 36)

In [17]:
%%time
pipeline.fit(X_train,y_train)

CPU times: user 7min 38s, sys: 2.48 s, total: 7min 40s
Wall time: 7min 41s


Pipeline(steps=[('vect',
                 CountVectorizer(token_pattern=None,
                                 tokenizer=<function tokenize at 0x7fe38811c790>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [30]:
y_pred = pipeline.predict(X_test)

In [31]:
y_pred.shape

(5244, 36)

In [32]:
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [33]:
labels = y_test.values
labels.shape

(5244, 36)

In [34]:
columns = y_test.columns #true labels to np array
columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [35]:
labels

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [38]:
from sklearn.metrics import classification_report

for i, col in enumerate(columns):
    print('Classification report for: ' + col)
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

Classification report for: related
              precision    recall  f1-score   support

           0       0.72      0.29      0.41      1230
           1       0.81      0.96      0.88      3977
           2       0.62      0.57      0.59        37

    accuracy                           0.80      5244
   macro avg       0.72      0.61      0.63      5244
weighted avg       0.79      0.80      0.77      5244

Classification report for: request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      4349
           1       0.89      0.44      0.59       895

    accuracy                           0.90      5244
   macro avg       0.89      0.72      0.77      5244
weighted avg       0.90      0.90      0.88      5244

Classification report for: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5222
           1       0.00      0.00      0.00        22

    accuracy             

/Users/isabellevea/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/isabellevea/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/isabellevea/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/isa

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5184
           1       0.00      0.00      0.00        60

    accuracy                           0.99      5244
   macro avg       0.49      0.50      0.50      5244
weighted avg       0.98      0.99      0.98      5244

Classification report for: other_infrastructure
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      5008
           1       0.00      0.00      0.00       236

    accuracy                           0.95      5244
   macro avg       0.48      0.50      0.49      5244
weighted avg       0.91      0.95      0.93      5244

Classification report for: weather_related
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      3801
           1       0.86      0.65      0.74      1443

    accuracy                           0.88      5244
   macro avg       0.87      0.81    

/Users/isabellevea/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/isabellevea/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/isabellevea/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [40]:
from sklearn.model_selection import GridSearchCV
#this is the default settings used in pipeline
pipeline.get_params()

{'memory': None,
 'steps': [('vect', CountVectorizer(token_pattern=None,
                   tokenizer=<function tokenize at 0x7f892913dea0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(token_pattern=None,
                 tokenizer=<function tokenize at 0x7f892913dea0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': None,
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__

In [44]:
from sklearn.model_selection import GridSearchCV
parameters =  {'clf__estimator__n_jobs':[2,4,6]
                   }

cv = GridSearchCV(pipeline, param_grid = parameters)


In [46]:
%%time
cv.fit(X_train, y_train)

CPU times: user 4min 56s, sys: 30.7 s, total: 5min 27s
Wall time: 40min 47s


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(token_pattern=None,
                                                        tokenizer=<function tokenize at 0x7f892913dea0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_jobs': [2, 4, 6]})

In [49]:
cv.best_params_

{'clf__estimator__n_jobs': 6}

In this pipeline, using n_jobs=6 is the best parameter

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [53]:
y_pred2 = cv.best_estimator_.predict(X_test)
for column in range(len(y_pred2.T)):
    # Print the classification report
    print(classification_report(labels[:, column], y_pred2[:, column]))

              precision    recall  f1-score   support

           0       0.72      0.27      0.40      1230
           1       0.81      0.97      0.88      3977
           2       0.68      0.51      0.58        37

    accuracy                           0.80      5244
   macro avg       0.74      0.58      0.62      5244
weighted avg       0.79      0.80      0.77      5244

              precision    recall  f1-score   support

           0       0.89      0.99      0.94      4349
           1       0.88      0.42      0.57       895

    accuracy                           0.89      5244
   macro avg       0.88      0.71      0.75      5244
weighted avg       0.89      0.89      0.88      5244

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5222
           1       0.00      0.00      0.00        22

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0

/Users/isabellevea/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/isabellevea/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/isabellevea/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/isa

              precision    recall  f1-score   support

           0       0.88      0.96      0.92      3801
           1       0.87      0.65      0.74      1443

    accuracy                           0.88      5244
   macro avg       0.87      0.81      0.83      5244
weighted avg       0.88      0.88      0.87      5244

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4830
           1       0.88      0.41      0.56       414

    accuracy                           0.95      5244
   macro avg       0.92      0.70      0.77      5244
weighted avg       0.95      0.95      0.94      5244

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      4755
           1       0.78      0.47      0.58       489

    accuracy                           0.94      5244
   macro avg       0.86      0.73      0.78      5244
weighted avg       0.93      0.94      0.93      5244

              preci

/Users/isabellevea/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/isabellevea/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/isabellevea/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [51]:
print(cv.best_params_)

{'clf__estimator__n_jobs': 6}


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF


In [52]:
#this is the default settings used in pipeline
pipeline.get_params()


{'memory': None,
 'steps': [('vect', CountVectorizer(token_pattern=None,
                   tokenizer=<function tokenize at 0x7f892913dea0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(token_pattern=None,
                 tokenizer=<function tokenize at 0x7f892913dea0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': None,
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__

In [55]:
#fine tuning parameters for tfidf
parameters_tfidf = {'tfidf__norm': ['l1', 'l2'],
                    'tfidf__smooth_idf': [True, False],
                    'tfidf__sublinear_tf': [True, False],
                    'tfidf__use_idf': [True, False]
                   }

cv_tfidf = GridSearchCV(pipeline, param_grid = parameters_tfidf)


In [56]:
%%time
cv_tfidf.fit(X_train, y_train)


CPU times: user 5h 55min 23s, sys: 3min 8s, total: 5h 58min 31s
Wall time: 6h 44min 55s


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(token_pattern=None,
                                                        tokenizer=<function tokenize at 0x7f892913dea0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'tfidf__norm': ['l1', 'l2'],
                         'tfidf__smooth_idf': [True, False],
                         'tfidf__sublinear_tf': [True, False],
                         'tfidf__use_idf': [True, False]})

In [ ]:
print(cv_tfidf.best_params_)

In [90]:
#new pipeline using the best parameters for tfidf
y_pred3 = cv_tfidf.best_estimator_.predict(X_test)

/Users/isabellevea/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [108]:
from sklearn.metrics import confusion_matrix, accuracy_score

for column in range(len(y_pred2.T)):
    # Print the classification report
    print(confusion_matrix(labels[:, column], y_pred2[:, column]))
    print(accuracy_score(labels[:, column], y_pred2[:, column]))
    print('Improvement of {:0.2f}%.'.format( 100 * 
                                            ((accuracy_score(labels[:, column], y_pred2[:, column]))-
                                             (accuracy_score(labels[:, column], y_pred[:, column])))/
                                            (accuracy_score(labels[:, column], y_pred[:, column]))))
    print()


[[ 443 1071   10]
 [ 161 4826    8]
 [   1   22   12]]
0.8057674702471773
Improvement of -0.08%.

[[5368   64]
 [ 642  480]]
0.8922795239548368
Improvement of -0.07%.

[[6531    0]
 [  23    0]]
0.996490692706744
Improvement of 0.00%.

[[3396  439]
 [1030 1689]]
0.7758620689655172
Improvement of -0.16%.

[[6017    5]
 [ 504   28]]
0.9223375038144644
Improvement of 0.00%.

[[6239    5]
 [ 293   17]]
0.9545315837656393
Improvement of 0.05%.

[[6373    3]
 [ 167   11]]
0.9740616417454989
Improvement of 0.05%.

[[6424    2]
 [ 127    1]]
0.9803173634421727
Improvement of 0.03%.

[[6325    7]
 [ 218    4]]
0.965669819957278
Improvement of -0.02%.

[[6554]]
1.0
Improvement of 0.00%.

[[6136    7]
 [ 318   93]]
0.9504119621605127
Improvement of -0.29%.

[[5802   47]
 [ 414  291]]
0.9296612755569118
Improvement of -0.81%.

[[5955   25]
 [ 438  136]]
0.9293561184009765
Improvement of -0.25%.

[[6450    0]
 [ 102    2]]
0.9844369850472994
Improvement of -0.09%.

[[6406    1]
 [ 143    4]]
0.9780

#### Fine tuning tf-idf part of the pipeline did not lead to subtantial accuracy improvement.


## What parameters to fine tune?

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

According to this article: https://towardsdatascience.com/random-forest-hyperparameters-and-how-to-fine-tune-them-17aee785ee0d the important parameters in RandomForestClassifier are:

- "The Nº of Decision Trees in the forest (in Scikit-learn this parameter is called n_estimators): The classification error decreases when we increase the number of trees." 
So we will test that by using 3 values including a small number and a large number.

- "The criteria with which to split on each node (Gini or Entropy for a classification task, or the MSE or MAE for regression)"

- "Number of random features to include at each node for splitting." 



In [63]:
print(pipeline.get_params().keys())

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estim

In [64]:
from sklearn.model_selection import GridSearchCV
n_estimators = [10, 100, 1000]
criterion = ['gini', 'entropy', 'log_loss']
max_features = ['sqrt', 'log2', None] # we can also look at int and float values but I decided not to to reduce the number of searches for this exercise.


parameters_clf = {'clf__estimator__n_estimators': n_estimators,
                  'clf__estimator__criterion': criterion,
                'clf__estimator__max_features': max_features}


cv_clf = GridSearchCV(pipeline, param_grid = parameters_clf)


In [66]:
%%time
cv_clf.fit(X_train, y_train)


KeyboardInterrupt: 

In [ ]:
print(cv_clf.best_params)

### 9. Export your model as a pickle file

In [ ]:
import pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(cv_clf, file)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

References I used for this notebook
https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

Model evaluation: https://medium.com/analytics-vidhya/evaluating-a-random-forest-model-9d165595ad56